In [27]:
import tensorflow as tf
from keras.models import load_model
from pathlib import Path
from art_api import utils, config
import os
from PIL import Image
import numpy as np
import pandas as pd

In [5]:
MODEL_PATH = Path("../models/vgg16_bal")
PATH_NGS = Path("../raw_data/aws10k")
PATH_NGS_SM = Path("../raw_data/aws10k_sm")

In [4]:
model = load_model(MODEL_PATH)

2022-09-17 13:41:49.242183: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
X, y = utils.load_data()

_request non-retriable exception: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object., 401
Traceback (most recent call last):
  File "/Users/enjiaochen/.pyenv/versions/lewagon/lib/python3.8/site-packages/gcsfs/retry.py", line 115, in retry_request
    return await func(*args, **kwargs)
  File "/Users/enjiaochen/.pyenv/versions/lewagon/lib/python3.8/site-packages/gcsfs/core.py", line 384, in _request
    validate_response(status, contents, path, args)
  File "/Users/enjiaochen/.pyenv/versions/lewagon/lib/python3.8/site-packages/gcsfs/retry.py", line 102, in validate_response
    raise HttpError(error)
gcsfs.retry.HttpError: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object., 401


HttpError: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object., 401

In [40]:
df = pd.read_csv(Path("../raw_data/data_raw_data_df_yourpaintings.csv"))

In [12]:
def resize_rescale(source):
    '''Download images and resize to 256x256 if not already in local disk
    Args:
        source: df, local, cloud
    Returns:
    '''
#    df = pd.read_csv(config.PATH_FILE)
    if source == "df":
        for index, row in df.iterrows():
            img_file = str(row["filename"])
            image = Image.open(os.path.join(config.PATH_YOURPAINTINGS, img_file))
            image = image.resize((256, 256), Image.ANTIALIAS)
            image.save(os.path.join(config.PATH_YOURPAINTINGS_SM, img_file))
            #imgs.append(np.array(image/255))
    if source == "path":
        for file in os.listdir(PATH_NGS):
            image = Image.open(os.path.join(PATH_NGS, file))
            image = image.resize((256, 256), Image.ANTIALIAS)
            image.save(os.path.join(PATH_NGS_SM, file))
    return "Images resized and rescaled"

In [41]:
resize_rescale(source="df")

/var/folders/11/0smvrnys6yn2p6mvqskf9vxm0000gn/T/ipykernel_12159/27729670.py:12: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize((256, 256), Image.ANTIALIAS)


'Images resized and rescaled'

In [18]:
def load_data():
    '''generates X and y'''
    '''read from disk or read from cloud <--- to be implemented'''
    
    imgs = []

    for file in os.listdir(PATH_NGS):
        image = Image.open(os.path.join(PATH_NGS_SM, file))   
        imgs.append(np.array(image))
        X = np.array(imgs)
        X.shape
    return X

In [21]:
X = load_data()
X = np.array(X)

In [20]:
X.shape

(10, 256, 256, 3)

In [22]:
pred = model.predict(X)

1/1 [==============================] - 2s 2s/step


In [24]:
pred[0]

array([0.24459508, 0.12576418, 0.06244959, 0.00660372, 0.04267118,
       0.01717522, 0.0298612 , 0.01034015, 0.08589564, 0.00666896],
      dtype=float32)

# Generate confusion matrix for model

In [37]:
df = pd.read_csv(Path("../raw_data/data_raw_data_df_yourpaintings.csv"))
df.columns

Index(['index', 'Image URL', 'Web page URL', 'Subset', 'Labels', 'filename',
       'labels', 'aeroplane', 'bird', 'boat', 'chair', 'cow', 'diningtable',
       'dog', 'horse', 'sheep', 'train'],
      dtype='object')

In [42]:
def load_data():
    '''generates X and y'''
    '''read from disk or read from cloud <--- to be implemented'''
    df = pd.read_csv(Path("../raw_data/data_raw_data_df_yourpaintings.csv"))
    df
    imgs = []
    for index, row in df.iterrows():
        img_file = str(row["filename"])
        image = Image.open(os.path.join(config.PATH_YOURPAINTINGS_SM, img_file))   
        imgs.append(np.array(image))
        X = np.array(imgs)
        X.shape
        y = df.drop(columns=['index', 'Image URL', 'Web page URL', 'Subset', 'Labels', 'filename', 'labels'])
        y.shape
    return X, y

In [ ]:
X, y = load_data()